# Experimento
## Avaliar o impacto da mudança de iterações no algoritmo de busca PSO


### Introdução
Como o espaço de busca dos parâmetros para cada técnica de Machine Learning costuma ser contínuo e de alta dimensionalidade, para atingir os valores ótimos é necessário uma busca exaustiva com alta granularidade, demandando assim um alto custo computacional, para resolver esse problema existem algumas técnicas que servem para realizar essa busca de maneira a diminuir os custos de tempo e processamento.


### Objetivo
Este experimento tem como objetivo verificar quais das funções disponíveis atingem o melhor resultado dentre as séries de retorno das ações componentes do índice IBOVESPA.

As técnicas a serem comparadas serão:
* Random Search.
* Grid Search.
* Particle Swarm Optimization (PSO).
* Nelder-Mead.
* Covariance Matrix Adaptation Evolution Strategy (CMA-ES).

### Metodologia

1. Primeiro passo é conseguir a base de dados, foi possível através do sistema do Yahoo! Finance conseguir alguns dos dados do IBOVESPA ( 43 dos 70 componentes ).

2. Transformar a série de preços em valores de retorno e criar uma matriz de padrões de entrada e saída, segundo o parâmetro de número de lags, para esse experimento foi escolhido o valor 4.

3. Para conseguir alguma relevância estatística serão feitas 50 execuções de um cross-validation para cada algoritmo, a variável a ser analisada será o valor da raiz quadrada do erro médio quadrático (RMSE) de cada iteração.

4. Será gerada uma matriz de 50 linhas e 5 colunas contendo todos os resultados obtidos, a primeira análise feita é se a distribuição dos valores de RMSE de cada técnica é igual, para tal foi usado um teste estatístico não-paramétrico, o **'Kruskal-Wallis H-test'**. Esse teste é o equivalente não-paramétrico do ANOVA e sua hipótese nula afirma que as distribuições dos grupos são iguais, se rejeitada é calculada uma matriz de 5 linhas e 5 colunas para verificar essas diferenças, cada elemento da matriz é o resultado de um teste não-paramétrico entre os pares de técnicas 'i' e 'j', o teste é o mesmo realizado anteriormente, porém quando o número de grupos é 2 (teste pareado) ele é conhecido por **'Mann–Whitney U test'**, equivalente não paramétrico do t-test.

5. O mesmo procedimento é feito para os tempos de execução.

6. Se as matrizes forem criadas, ou seja, existe diferença estatística entre os algoritmos de busca, é calculado um vetor com a soma de cada linha da matriz para criar um ranking dessa técnica em relação as outras. 

Por exemplo, temos uma matriz formada por 0's e 1's, onde uma linha representa o desempenho de uma técnica em relação as outras, se for 0 temos a possibilidade de que as técnicas não apresentam diferença estatística entre si ou então o i-ésimo algoritmo é melhor que o j-ésimo ( apresenta um valor médio menor, nesse caso melhor pois tratamos de erro ), se for 1 só existe a possibilidade que existe diferença estatística entre as técnicas e que a i-ésima apresenta um pior desempenho ( valor médio maior ).

Então cada elemento do vetor varia entre 0 e 4, onde o primeiro representa que aquele algoritmo não foi pior que nenhum outro e 4 indica que ele foi pior que todos os outros.

Esse experimento foi feito utilizando 3 diferentes regressores, para verificar se os resultados eram consistentes entre diferentes técnicas de Machine Learning, foram escolhidos o **elmK**, **elmR** e **svr**.

### Descrição